In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("mxmh.csv")

In [3]:
df.head()

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,target,Permissions
0,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,No,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.


In [4]:
df['target'].value_counts()

Improve      542
No effect    169
Worsen        17
Name: target, dtype: int64

In [5]:
df.drop("Timestamp", axis=1, inplace=True)

In [6]:
df = df.dropna()

In [7]:
df.shape

(616, 32)

In [8]:
df.duplicated().sum()

0

In [9]:
def one_hot(data):
    if 'object' in [data[i].dtype for i in data]:
        object_data = data.select_dtypes(include='object')
        object_df = pd.get_dummies(object_data)
        rest_data = data.select_dtypes(exclude='object')
        return pd.concat([object_df, rest_data],axis=1)
    else:
        return data

In [10]:
def segment_production_runs(data, sample_size, n=3):
    production_data = dict()
    for i in range(n):
        production_data[i+1] = data.sample(sample_size, random_state=42)
        remove_data = production_data[i+1].index.tolist()
        data = data.drop(remove_data,axis=0)
    return data, production_data

In [11]:
baseline, production_runs = segment_production_runs(df, 100)

In [12]:
baseline.shape

(316, 32)

In [13]:
for i in production_runs:
    production_runs[i].to_csv(f"production_runs{i}.csv")

In [14]:
baseline.head()

,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,BPM,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,target,Permissions
6,18.0,YouTube Music,3.0,Yes,Yes,No,Video game music,Yes,Yes,66.0,...,Rarely,Never,Never,Sometimes,4.0,8.0,6.0,0.0,Improve,I understand.
7,21.0,Spotify,1.0,Yes,No,No,K pop,Yes,Yes,95.0,...,Sometimes,Rarely,Never,Rarely,5.0,3.0,5.0,3.0,Improve,I understand.
9,18.0,I do not use a streaming service.,1.0,Yes,No,No,R&B,Yes,Yes,155.0,...,Sometimes,Rarely,Sometimes,Sometimes,2.0,2.0,5.0,1.0,Improve,I understand.
13,19.0,Spotify,2.0,Yes,No,No,Country,Yes,No,88.0,...,Never,Very frequently,Never,Never,2.0,1.0,2.0,0.0,Improve,I understand.
19,15.0,Spotify,3.0,No,No,No,Hip hop,No,No,120.0,...,Sometimes,Very frequently,Rarely,Never,7.0,3.0,0.0,2.0,Improve,I understand.


In [15]:
one_base = one_hot(baseline.drop("target", axis=1))

In [16]:
one_base['target'] = baseline['target']

In [17]:
one_base.head()

,Primary streaming service_Apple Music,Primary streaming service_I do not use a streaming service.,Primary streaming service_Other streaming service,Primary streaming service_Pandora,Primary streaming service_Spotify,Primary streaming service_YouTube Music,While working_No,While working_Yes,Instrumentalist_No,Instrumentalist_Yes,...,Frequency [Video game music]_Very frequently,Permissions_I understand.,Age,Hours per day,BPM,Anxiety,Depression,Insomnia,OCD,target
6,0,0,0,0,0,1,0,1,0,1,...,0,1,18.0,3.0,66.0,4.0,8.0,6.0,0.0,Improve
7,0,0,0,0,1,0,0,1,1,0,...,0,1,21.0,1.0,95.0,5.0,3.0,5.0,3.0,Improve
9,0,1,0,0,0,0,0,1,1,0,...,0,1,18.0,1.0,155.0,2.0,2.0,5.0,1.0,Improve
13,0,0,0,0,1,0,0,1,1,0,...,0,1,19.0,2.0,88.0,2.0,1.0,2.0,0.0,Improve
19,0,0,0,0,1,0,1,0,1,0,...,0,1,15.0,3.0,120.0,7.0,3.0,0.0,2.0,Improve


In [18]:
X = one_base.drop("target", axis=1)
y = one_base['target']

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [22]:
scaler = StandardScaler()

In [23]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
model_knn = KNeighborsClassifier(n_neighbors=15)

In [26]:
model_knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=15)

In [27]:
y_pred = model_knn.predict(X_test)

In [70]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score

In [29]:
accuracy_score(y_test, y_pred)

0.7142857142857143

In [30]:
def EncodeLabels(gt_label, pd_label):
    
    label = LabelEncoder()
    gt = label.fit_transform(gt_label)
    pd = label.transform(pd_label)
    return gt, pd

In [31]:
def classification_metrics(true_labels,predicted_labels):
    metrics = dict()
    metrics['Accuracy'] = np.round(accuracy_score(true_labels, predicted_labels)*100, decimals=2)
    metrics['Precision'] = np.round(precision_score(true_labels, predicted_labels, average="weighted")*100, decimals=2)
    metrics['Recall'] = np.round(recall_score(true_labels, predicted_labels, average="weighted")*100, decimals=2)
    metrics['F1'] = np.round(f1_score(true_labels, predicted_labels, average="weighted")*100, decimals=2)
    
    en_true_labels, en_predicted_labels = EncodeLabels(true_labels, predicted_labels)
    
    metrics['ROC_AUC'] = np.round(roc_auc_score(np.argmax(en_true_labels, axis = 0), np.argmax(en_predicted_labels, axis = 0), multi_class='ovr')*100, decimals=2)
    cm = confusion_matrix(true_labels, predicted_labels)
    FP = np.sum(cm, axis=0) - np.diag(cm)  # False positives
    TN = np.sum(cm) - np.sum(cm, axis=0) - np.sum(cm, axis=1) + np.diag(cm)  # True negatives

# Calculate the total number of false positives and true negatives
    total_FP = np.sum(FP)
    total_TN = np.sum(TN)
    
    # metrics['True Positive Rate'] = true_positive / (true_positive + false_negative)
    metrics['False Positive Rate'] = np.round((total_FP / (total_FP + total_TN))*100, decimals=2)
    return metrics

In [32]:
classification_metrics(y_test, y_pred)

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TypeError: Singleton array array(25, dtype=int64) cannot be considered a valid collection.

In [35]:
len(y_pred)

105

In [36]:
y_test.value_counts()

Improve      75
No effect    25
Worsen        5
Name: target, dtype: int64

In [40]:
y_test_en, y_pred_en = EncodeLabels(y_test, y_pred
                                )

In [110]:
roc_auc_score(y_, y_p, multi_class="ovr")

0.5

In [44]:
y_test_en[0]

0

In [45]:
y_test_en

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [48]:
from sklearn.preprocessing import LabelBinarizer

In [49]:
lb = LabelBinarizer()

In [50]:
y_ = lb.fit_transform(y_test)

In [51]:
y_

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0,

In [52]:
y_p = lb.transform(y_pred)

In [53]:
y_p

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,

In [56]:
y_p[:, 1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
lb.classes_

array(['Improve', 'No effect', 'Worsen'], dtype='<U9')

In [61]:
len(y_test.unique()) > 2

True

In [127]:
def plot_PR(true_labels, predicted_labels):
    
    if len(true_labels.unique()) > 2 : 
        lb = LabelBinarizer()
        true_m = lb.fit_transform(true_labels)
        pred_m = lb.transform(predicted_labels)
    
        precision, recall, average_precision = dict(), dict(), dict()
        
        for i in range(len(lb.classes_)):
            precision[i], recall[i], _ = precision_recall_curve(true_m[:, i], pred_m[:, i])
            average_precision[lb.classes_[i]] = average_precision_score(true_m[:, i], pred_m[:, i])

        figure = px.area(x=[i[0] for i in list(recall.values())], y=[i[0] for i in list(precision.values())],
            title=f'Precision-Recall Curve {average_precision}',
            labels=dict(x='Recall', y='Precision'),
            width=700, height=500)
        figure.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=1, y1=0)
        
        return figure, precision, recall

    else:
        
        precision, recall, _ = precision_recall_curve(true_labels, predicted_labels)
        precision_ = np.round(precision_score(true_labels, predicted_labels)*100, decimals=2)
        recall_ = np.round(recall_score(true_labels, predicted_labels)*100, decimals=2)
        figure = px.area(x=recall, y=precision,
            title=f'Precision-Recall Curve',
            labels=dict(x='Recall', y='Precision'),
            width=700, height=500)
        figure.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=1, y1=0)
        return figure, precision_, recall_

In [128]:
figure, precision, recall, average_precision, df = plot_PR(y_test, y_pred)

In [104]:
recall[0][0]

1.0

In [95]:
import plotly.express as px

In [129]:
figure

In [88]:
[i[0] for i in list(precision.values())]

[0.7142857142857143, 0.23809523809523808, 0.047619047619047616]

In [82]:
recall

{0: array([1., 0.]), 1: array([1., 0.]), 2: array([1., 0.])}

In [83]:
average_precision

{0: 0.7142857142857143, 1: 0.23809523809523808, 2: 0.047619047619047616}

In [102]:
df

,Precision,Recall,Class
0,0.714286,1.0,Improve
1,1.000000,0.0,Improve
2,0.238095,1.0,No effect
3,1.000000,0.0,No effect
4,0.047619,1.0,Worsen
5,1.000000,0.0,Worsen


In [120]:
def plot_roc(true_labels, predicted_labels):
    
    if len(true_labels.unique()) > 2 : 
        lb = LabelBinarizer()
        true_m = lb.fit_transform(true_labels)
        pred_m = lb.transform(predicted_labels)
        
        
        fpr, tpr, roc_auc = dict(), dict(), dict()
        for i in range(len(lb.classes_)):
            fpr[i], tpr[i], _ = roc_curve(true_m[:, i], pred_m[:, i])
            roc_auc[lb.classes_[i]] = auc(fpr[i], tpr[i])

        figure = px.area(x=fpr, y=tpr,
            title=f'ROC Curve (AUC={roc_auc})',
            labels=dict(x='False Positive Rate', y='True Positive Rate'),
            width=700, height=500)
        figure.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1)
        return figure, roc_auc
    
    else:
        
        false_positve_r, true_positive_r, thresholds = roc_curve(true_labels, predicted_labels)
        roc_auc = auc(false_positve_r, true_positive_r)
        figure = px.area(x=false_positve_r, y=true_positive_r,
            title=f'ROC Curve (AUC={roc_auc})',
            labels=dict(x='False Positive Rate', y='True Positive Rate'),
            width=700, height=500)
        figure.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1)
        return figure, roc_auc

In [121]:
figure, roc_auc = plot_roc(y_test, y_pred)

In [122]:
figure

In [119]:
lb.classes_

array(['Improve', 'No effect', 'Worsen'], dtype='<U9')

In [130]:
def plot_confusion_matrix(true_labels,predicted_labels, class_labels):
    
    cm_df = pd.DataFrame(confusion_matrix(true_labels, predicted_labels), index=class_labels, columns=class_labels)
    fig = px.imshow(cm_df,
                    labels=dict(x="Predicted", y="True", color="Count"),
                    x=class_labels,
                    y=class_labels,
                    text_auto=True)
    return fig

In [132]:
plot_confusion_matrix(y_test, y_pred, list(y_test.unique()))

In [133]:
def classification_metrics(true_labels,predicted_labels):

    metrics = dict()
    
    if len(true_labels.unique()) > 2 : 
        lb = LabelBinarizer()
        true_m = lb.fit_transform(true_labels)
        pred_m = lb.transform(predicted_labels)
        
        metrics['Accuracy'] = np.round(accuracy_score(true_labels, predicted_labels)*100, decimals=2)
        metrics['Precision'] = np.round(precision_score(true_m, pred_m, average="weighted")*100, decimals=2)
        metrics['Recall'] = np.round(recall_score(true_m, pred_m, average="weighted")*100, decimals=2)
        metrics['F1'] = np.round(f1_score(true_m, pred_m, average="weighted")*100, decimals=2)
        metrics['ROC_AUC'] = np.round(roc_auc_score(true_m, pred_m, multi_class="ovr")*100, decimals=2)
        
        cm = confusion_matrix(true_labels, predicted_labels)
        FP = np.sum(cm, axis=0) - np.diag(cm)
        TN = np.sum(cm) - np.sum(cm, axis=0) - np.sum(cm, axis=1) + np.diag(cm)

        total_FP = np.sum(FP)
        total_TN = np.sum(TN)
    
        metrics['False Positive Rate'] = np.round((total_FP / (total_FP + total_TN))*100, decimals=2)
        
        return metrics
    
    else:
        
        metrics['Accuracy'] = np.round(accuracy_score(true_labels, predicted_labels)*100, decimals=2)
        metrics['Precision'] = np.round(precision_score(true_labels, predicted_labels)*100, decimals=2)
        metrics['Recall'] = np.round(recall_score(true_labels, predicted_labels)*100, decimals=2)
        metrics['F1'] = np.round(f1_score(true_labels, predicted_labels)*100, decimals=2)
        metrics['ROC_AUC'] = np.round(roc_auc_score(true_labels, predicted_labels)*100, decimals=2)
        true_negative, false_positive, false_negative, true_positive = confusion_matrix(true_labels, predicted_labels).ravel()
        # metrics['True Positive Rate'] = true_positive / (true_positive + false_negative)
        metrics['False Positive Rate'] = np.round((false_positive / (false_positive + true_negative))*100, decimals=2)
        
        return metrics

In [134]:
classification_metrics(y_test, y_pred)

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



{'Accuracy': 71.43,
 'Precision': 51.02,
 'Recall': 71.43,
 'F1': 59.52,
 'ROC_AUC': 50.0,
 'False Positive Rate': 14.29}

In [136]:
model_knn

KNeighborsClassifier(n_neighbors=15)

In [137]:
import joblib

In [138]:
joblib.dump(model_knn, "model.joblib")

['model.joblib']

In [165]:
prod2 = pd.read_csv("production_runs3.csv")

In [166]:
prod2.head()

,Unnamed: 0,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,target,Permissions
0,283,22.0,Spotify,6.0,Yes,No,No,Jazz,Yes,Yes,...,Sometimes,Rarely,Very frequently,Sometimes,8.0,8.0,7.0,1.0,Improve,I understand.
1,538,24.0,Spotify,6.0,Yes,Yes,Yes,Rock,Yes,Yes,...,Never,Rarely,Very frequently,Sometimes,8.0,8.0,10.0,0.0,Improve,I understand.
2,727,26.0,YouTube Music,1.0,Yes,No,No,Metal,Yes,Yes,...,Never,Never,Never,Rarely,0.0,0.0,0.0,0.0,No effect,I understand.
3,173,28.0,Spotify,2.0,No,No,Yes,Metal,No,No,...,Never,Never,Sometimes,Rarely,1.0,7.0,5.0,1.0,Improve,I understand.
4,134,21.0,Spotify,3.0,Yes,Yes,No,Rock,Yes,No,...,Never,Rarely,Very frequently,Sometimes,8.0,9.0,2.0,0.0,Improve,I understand.


In [167]:
prod2.drop("Unnamed: 0", inplace=True, axis=1)

In [168]:
prod2.head()

,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,BPM,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,target,Permissions
0,22.0,Spotify,6.0,Yes,No,No,Jazz,Yes,Yes,140.0,...,Sometimes,Rarely,Very frequently,Sometimes,8.0,8.0,7.0,1.0,Improve,I understand.
1,24.0,Spotify,6.0,Yes,Yes,Yes,Rock,Yes,Yes,118.0,...,Never,Rarely,Very frequently,Sometimes,8.0,8.0,10.0,0.0,Improve,I understand.
2,26.0,YouTube Music,1.0,Yes,No,No,Metal,Yes,Yes,136.0,...,Never,Never,Never,Rarely,0.0,0.0,0.0,0.0,No effect,I understand.
3,28.0,Spotify,2.0,No,No,Yes,Metal,No,No,146.0,...,Never,Never,Sometimes,Rarely,1.0,7.0,5.0,1.0,Improve,I understand.
4,21.0,Spotify,3.0,Yes,Yes,No,Rock,Yes,No,160.0,...,Never,Rarely,Very frequently,Sometimes,8.0,9.0,2.0,0.0,Improve,I understand.


In [169]:
prod2.drop("target", inplace=True, axis=1)

In [170]:
df = one_hot(prod2)

In [171]:
df.head()

,Primary streaming service_Apple Music,Primary streaming service_I do not use a streaming service.,Primary streaming service_Other streaming service,Primary streaming service_Pandora,Primary streaming service_Spotify,Primary streaming service_YouTube Music,While working_No,While working_Yes,Instrumentalist_No,Instrumentalist_Yes,...,Frequency [Video game music]_Sometimes,Frequency [Video game music]_Very frequently,Permissions_I understand.,Age,Hours per day,BPM,Anxiety,Depression,Insomnia,OCD
0,0,0,0,0,1,0,0,1,1,0,...,1,0,1,22.0,6.0,140.0,8.0,8.0,7.0,1.0
1,0,0,0,0,1,0,0,1,0,1,...,1,0,1,24.0,6.0,118.0,8.0,8.0,10.0,0.0
2,0,0,0,0,0,1,0,1,1,0,...,0,0,1,26.0,1.0,136.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,0,1,0,1,0,...,0,0,1,28.0,2.0,146.0,1.0,7.0,5.0,1.0
4,0,0,0,0,1,0,0,1,0,1,...,1,0,1,21.0,3.0,160.0,8.0,9.0,2.0,0.0


In [162]:
scaler

StandardScaler()

In [148]:
joblib.dump(scaler, "scaler.joblib")

['scaler.joblib']

In [172]:
another = scaler.transform(df)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Fav genre_Gospel
- Fav genre_Latin


In [164]:
pr1_pred = model_knn.predict(another)

In [153]:
prod1['target'] = pr1_pred

In [154]:
prod1.to_csv("prod1.csv")

In [173]:
prod1.head()

,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,BPM,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Permissions,target
0,43.0,I do not use a streaming service.,6.0,Yes,Yes,Yes,Video game music,Yes,Yes,109.0,...,Rarely,Never,Sometimes,Very frequently,5.0,4.0,8.0,3.0,I understand.,Improve
1,16.0,Spotify,4.0,Yes,No,Yes,Lofi,Yes,No,85.0,...,Never,Rarely,Rarely,Very frequently,7.0,1.0,3.0,5.0,I understand.,Improve
2,18.0,Apple Music,2.0,Yes,No,No,R&B,Yes,No,80.0,...,Very frequently,Very frequently,Never,Never,7.0,5.0,5.0,5.0,I understand.,Improve
3,20.0,Spotify,3.0,Yes,No,No,Rock,Yes,Yes,120.0,...,Rarely,Rarely,Very frequently,Sometimes,8.0,5.0,2.0,3.0,I understand.,Improve
4,23.0,Spotify,4.0,Yes,No,No,Hip hop,Yes,Yes,78.0,...,Very frequently,Very frequently,Sometimes,Sometimes,7.0,6.0,7.0,5.0,I understand.,Improve


In [177]:
model_knn.n_neighbors

15

In [179]:
model_knn.predict_proba(scaler.transform(one_hot(prod1.drop("target", axis=1))))

array([[0.86666667, 0.13333333, 0.        ],
       [0.8       , 0.13333333, 0.06666667],
       [0.66666667, 0.26666667, 0.06666667],
       [0.86666667, 0.13333333, 0.        ],
       [0.86666667, 0.06666667, 0.06666667],
       [0.66666667, 0.33333333, 0.        ],
       [0.66666667, 0.33333333, 0.        ],
       [0.86666667, 0.13333333, 0.        ],
       [0.66666667, 0.26666667, 0.06666667],
       [0.66666667, 0.33333333, 0.        ],
       [0.73333333, 0.26666667, 0.        ],
       [0.66666667, 0.26666667, 0.06666667],
       [0.93333333, 0.06666667, 0.        ],
       [0.73333333, 0.2       , 0.06666667],
       [0.8       , 0.2       , 0.        ],
       [0.73333333, 0.2       , 0.06666667],
       [0.6       , 0.33333333, 0.06666667],
       [0.33333333, 0.66666667, 0.        ],
       [0.8       , 0.2       , 0.        ],
       [0.8       , 0.2       , 0.        ],
       [0.6       , 0.4       , 0.        ],
       [0.73333333, 0.2       , 0.06666667],
       [0.

In [180]:
a = [1,2,3,4]
a.index(1)

0

In [181]:
[1,2] in a

False

In [182]:
baseline.to_csv("baseline.csv")

In [183]:
lb.classes_[0]

'Improve'